In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
%matplotlib inline
train_bankcard= pd.read_csv(r'AI_risk_train_V3.0\train_bankcard_info.csv')
test_bankcard= pd.read_csv(r'AI_risk_test_V3.0\test_bankcard_info.csv')
merge_df = pd.concat([train_bankcard,test_bankcard],axis=0)
merge_df.head()

,bank_name,tail_num,card_type,phone,id
0,中信银行,9753,储蓄卡,177****8427,585401206734589960
1,农业银行,2174,储蓄卡,150****4551,587694980785508360
2,中国交通银行,4832,信用卡,185****1989,602098357531316232
3,北京银行,5697,储蓄卡,186****1634,567205103669350408
4,中信银行,7703,储蓄卡,158****7655,577538085852549384


In [26]:
merge_df.tail(10)

,bank_name,tail_num,card_type,phone,id
96139,建设银行,9635,储蓄卡,136****6697,630245723375407112
96140,中国交通银行,1737,信用卡,159****8582,625057981578350600
96141,ccb,5097,储蓄卡,135****9912,628031341115478280
96142,建设银行,5100,储蓄卡,135****6922,632279204599173384
96143,ccb,7184,信用卡,135****7233,636281816289906952
96144,招商银行,3010,信用卡,151****1625,623581591351857160
96145,工商银行,7274,储蓄卡,135****0463,630240471955804424
96146,农业银行,1779,储蓄卡,138****0353,634962681584947208
96147,建设银行,6212,储蓄卡,139****7652,628987541764182024
96148,建设银行,4580,储蓄卡,152****4158,638171158235058440


In [27]:
merge_df['bank_phone'] = merge_df.phone
merge_df.pop('phone')
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411638 entries, 0 to 96148
Data columns (total 5 columns):
bank_name     411331 non-null object
tail_num      411638 non-null int64
card_type     411638 non-null object
id            411638 non-null object
bank_phone    411638 non-null object
dtypes: int64(1), object(4)
memory usage: 18.8+ MB


字段名	字段解释
id	申请贷款唯一编号
bank_name	银行名称
tail_num	银行卡号后四位
card_type	银行卡类型
phone	银行卡绑定手机号(脱敏)


In [28]:
vars_with_missing = []

for f in merge_df.columns:
    missings = merge_df[f].isnull().sum()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/merge_df.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))

print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

Variable bank_name has 307 records (0.07%) with missing values
In total, there are 1 variables with missing values


In [29]:
for f in merge_df.columns:
    dist_values = merge_df[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values)) 

Variable bank_name has 196 distinct values
Variable tail_num has 10000 distinct values
Variable card_type has 2 distinct values
Variable id has 167960 distinct values
Variable bank_phone has 124952 distinct values


添加每个id绑定的银行卡数
===

In [30]:
card_counts = merge_df.id.value_counts().to_frame(name='card_counts').reset_index()
card_counts.columns=['id','card_counts']
card_counts.describe()

,card_counts
count,167960.000000
mean,2.450810
std,2.364677
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,42.000000


In [31]:
df = merge_df[['id','bank_phone']].groupby(['id'])['bank_phone'].apply(lambda x:len(set(x))).reset_index()
df.columns=['id','bank_phone_counts']
df.describe()

,bank_phone_counts
count,167960.000000
mean,1.091236
std,0.323524
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,8.000000


In [32]:
df = pd.merge(card_counts,df,how='right',on='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167960 entries, 0 to 167959
Data columns (total 3 columns):
id                   167960 non-null object
card_counts          167960 non-null int64
bank_phone_counts    167960 non-null int64
dtypes: int64(2), object(1)
memory usage: 5.1+ MB


In [33]:
tail_num = merge_df[['id','tail_num']].groupby(['id'])['tail_num'].apply(lambda x:len(set(x))).reset_index()
tail_num.columns=['id','tail_num_counts']
tail_num.describe()

,tail_num_counts
count,167960.000000
mean,2.114218
std,1.710564
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,33.000000


In [34]:
df = pd.merge(tail_num,df,how='right',on='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167960 entries, 0 to 167959
Data columns (total 4 columns):
id                   167960 non-null object
tail_num_counts      167960 non-null int64
card_counts          167960 non-null int64
bank_phone_counts    167960 non-null int64
dtypes: int64(3), object(1)
memory usage: 6.4+ MB


In [35]:
bank_name = merge_df[['id','bank_name']].groupby(['id'])['bank_name'].apply(lambda x:len(set(x))).reset_index()
bank_name.columns=['id','bank_name_counts']
bank_name.describe()

,bank_name_counts
count,167960.000000
mean,2.136366
std,1.822336
min,1.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,28.000000


In [36]:
df = pd.merge(bank_name,df,how='right',on='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167960 entries, 0 to 167959
Data columns (total 5 columns):
id                   167960 non-null object
bank_name_counts     167960 non-null int64
tail_num_counts      167960 non-null int64
card_counts          167960 non-null int64
bank_phone_counts    167960 non-null int64
dtypes: int64(4), object(1)
memory usage: 7.7+ MB


In [37]:
card_type = merge_df[['id','card_type']].groupby(['id'])['card_type'].apply(lambda x:len(set(x))).reset_index()
card_type.columns=['id','card_type_counts']
card_type.describe()

,card_type_counts
count,167960.000000
mean,1.318165
std,0.465765
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


In [38]:
card_type.card_type_counts = card_type.card_type_counts.map({1:0,2:1})
card_type.describe()

,card_type_counts
count,167960.000000
mean,0.318165
std,0.465765
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [39]:
df = pd.merge(card_type,df,how='right',on='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167960 entries, 0 to 167959
Data columns (total 6 columns):
id                   167960 non-null object
card_type_counts     167960 non-null int64
bank_name_counts     167960 non-null int64
tail_num_counts      167960 non-null int64
card_counts          167960 non-null int64
bank_phone_counts    167960 non-null int64
dtypes: int64(5), object(1)
memory usage: 9.0+ MB


In [40]:
merge_df.bank_name.value_counts()

建设银行          76406
工商银行          57187
农业银行          50082
招商银行          20989
ccb           19498
中国银行          19190
中信银行          16979
中国交通银行        14996
icbc          14711
中国光大银行        13198
邮政储蓄          12835
abc           11848
中国民生银行         9246
平安银行           8412
广发银行           8368
浦发银行           7374
cmb            5769
citic          5358
boc            4963
ceb            4547
兴业银行           4370
bcom           3981
post           3253
gdb            2975
pab            2671
cmbc           2198
华夏银行           2061
spdb           2013
cib             996
hxb             603
              ...  
临商银行              2
营口银行              2
boyk              2
烟台银行              2
吴江农村商业银行          2
zjtlcb            2
bojl              2
威海市商业银行           2
东营银行              2
wzcb              2
郑州银行              1
莱商银行              1
gdnycb            1
铁岭银行              1
cbhb              1
盘锦银行              1
gzrcc             1
平顶山银行             1
天津滨海农村商业银行        1


In [41]:
def tranf(x):
    x = str(x)
    if x.startswith(('建设','ccb')):
        x= 'a'
    elif x.startswith(('工商','icbc')):
        x = 'b'
    elif x.startswith(('农业','abc')):
        x = 'c'
    elif x.startswith(('招商','cmb')):
        x = 'd' 
    elif x.startswith(('中国银行','boc')):
        x = 'e'
    elif x.startswith(('中信银行','ccb','citic')):
        x = 'f'
    elif x.startswith(('中国交通','bocom')):
        x = 'g'
    elif x.startswith(('中国光大','ceb')):
        x = 'h'
    elif x.startswith(('邮政储蓄','psbc','post')):
        x = 'i'
    elif x.startswith(('中国民生','cmbc')):
        x = 'j'
    elif x.startswith(('广发','gdb')):
        x = 'k'
    elif x.startswith(('平安')):
        x = 'l'
    elif x.startswith(('兴业','cib')):
        x = 'm'
    elif x.startswith(('浦发','spdb')):
        x = 'n'
    else :
        x ='o'
    return x
merge_df['bank_name'] = merge_df['bank_name'].apply(tranf)
merge_df['bank_name'].value_counts()

a    95904
b    71898
c    61930
d    28956
e    24163
f    22337
h    17745
i    16088
g    14996
o    13867
k    11343
n     9387
j     9246
l     8412
m     5366
Name: bank_name, dtype: int64

In [42]:
merge_df.card_type = merge_df.card_type.map({'储蓄卡':0,'信用卡':1})
merge_df.head()

,bank_name,tail_num,card_type,id,bank_phone
0,f,9753,0,585401206734589960,177****8427
1,c,2174,0,587694980785508360,150****4551
2,g,4832,1,602098357531316232,185****1989
3,o,5697,0,567205103669350408,186****1634
4,f,7703,0,577538085852549384,158****7655


In [43]:
merge_df_later = merge_df.groupby('id',as_index=False).tail(1)
merge_df_later.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167960 entries, 0 to 96148
Data columns (total 5 columns):
bank_name     167960 non-null object
tail_num      167960 non-null int64
card_type     167960 non-null int64
id            167960 non-null object
bank_phone    167960 non-null object
dtypes: int64(2), object(3)
memory usage: 7.7+ MB


In [44]:
merge_df_later = pd.merge(merge_df_later,df,how='left',on='id')
merge_df_later.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167960 entries, 0 to 167959
Data columns (total 10 columns):
bank_name            167960 non-null object
tail_num             167960 non-null int64
card_type            167960 non-null int64
id                   167960 non-null object
bank_phone           167960 non-null object
card_type_counts     167960 non-null int64
bank_name_counts     167960 non-null int64
tail_num_counts      167960 non-null int64
card_counts          167960 non-null int64
bank_phone_counts    167960 non-null int64
dtypes: int64(7), object(3)
memory usage: 14.1+ MB


In [45]:
columns_object = ['bank_name']
merge_df_later = pd.get_dummies(merge_df_later, columns=columns_object, drop_first=True)
merge_df_later.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167960 entries, 0 to 167959
Data columns (total 23 columns):
tail_num             167960 non-null int64
card_type            167960 non-null int64
id                   167960 non-null object
bank_phone           167960 non-null object
card_type_counts     167960 non-null int64
bank_name_counts     167960 non-null int64
tail_num_counts      167960 non-null int64
card_counts          167960 non-null int64
bank_phone_counts    167960 non-null int64
bank_name_b          167960 non-null uint8
bank_name_c          167960 non-null uint8
bank_name_d          167960 non-null uint8
bank_name_e          167960 non-null uint8
bank_name_f          167960 non-null uint8
bank_name_g          167960 non-null uint8
bank_name_h          167960 non-null uint8
bank_name_i          167960 non-null uint8
bank_name_j          167960 non-null uint8
bank_name_k          167960 non-null uint8
bank_name_l          167960 non-null uint8
bank_name_m          167960

In [46]:
merge_df_later.pop('tail_num')
merge_df_later.to_csv(r'Datapre\train_bankcard.csv',index=False,encoding='utf-8')
df1 =pd.read_csv(r'Datapre\train_bankcard.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167960 entries, 0 to 167959
Data columns (total 22 columns):
card_type            167960 non-null int64
id                   167960 non-null object
bank_phone           167960 non-null object
card_type_counts     167960 non-null int64
bank_name_counts     167960 non-null int64
tail_num_counts      167960 non-null int64
card_counts          167960 non-null int64
bank_phone_counts    167960 non-null int64
bank_name_b          167960 non-null int64
bank_name_c          167960 non-null int64
bank_name_d          167960 non-null int64
bank_name_e          167960 non-null int64
bank_name_f          167960 non-null int64
bank_name_g          167960 non-null int64
bank_name_h          167960 non-null int64
bank_name_i          167960 non-null int64
bank_name_j          167960 non-null int64
bank_name_k          167960 non-null int64
bank_name_l          167960 non-null int64
bank_name_m          167960 non-null int64
bank_name_n          167960

E:\soft-all\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
